In [ ]:
%load_ext autoreload
%autoreload 2
%pylab inline

In [ ]:
%load_ext Cython
%load_ext line_profiler
%load_ext memory_profiler

## Import framework main libs

In [ ]:
from tmqrfeed.manager import DataManager
from tmqrstrategy.optimizers import OptimizerBase, OptimizerGenetic


## Import strategy from module

In [ ]:
from tmqralphas.strategy_dsp_lpbp_combination import Strategy_DSP_LPBP_Combination

# Strategy inheritance trick!

We inherited Strategy_DSP_LPBP_Combination strategy the logic remains the same, but position management in different.

In [ ]:
import pandas as pd
from tmqrfeed.quotes import QuoteIndex
from tmqr.logs import log
from tmqr.errors import PositionNotFoundError


class Strategy_DSP_LPBP_ManualPositionHedge(Strategy_DSP_LPBP_Combination):    
    #
    # Manually compose position of alpha
    #
    def calculate_position(self, date: datetime, exposure_record: pd.DataFrame):

        # We are calling sum() because we have multiple records of 'exposure'
        # 1-alpha member of best in the swarm per row
        if 'exposure' not in exposure_record:
            raise StrategyError(
                "'exposure_record' expected to have 'exposure' column, check alpha's calculate(...) method "
                "to make sure that it returns valid pandas.DataFrame with exposure column or just check "
                "for 'return self.exposure(...)' in the last line")
        exposure = exposure_record['exposure'].sum()
        
        #
        # Keep previous position
        #     Otherwise you have to construct position from scratch at the beginning of each day
        self.position.keep_previous_position(date)
        
        # You MUST also manage expired position 
        if self.position.almost_expired_ratio(date) > 0:
            self.position.close(date)
        
        if not self.position.has_position(date):
            if exposure != 0:
                # Getting instrument name from index name 
                # 'US.ES_ContFutEOD' -> 'US.ES'
                instrument_name = self.context['index_name'].split('_')[0]
                
                fut, opt_chain = self.dm.chains_options_get(instrument_name, date)
                
                # set the primary position
                self.position.add_transaction(date, fut, exposure)        
                
                
                if exposure > 0:
                    # Hedgind long alphas exposures by long put
                    #self.position.add_transaction(date, opt_chain.find(date, 0.25, 'P', how='delta'), -3.0 * exposure) 
                    self.position.add_transaction(date, opt_chain.find(date, 0.55, 'P', how='delta'), -1.0 * exposure) 
                elif exposure < 0:
                    # Hedgind short alphas exposures by long put
                    self.position.add_transaction(date, opt_chain.find(date, 0.15, 'C', how='delta'), -2.0 * exposure)        
        else:
            if exposure == 0:
                # Close all
                self.position.close(date)
    

In [ ]:
ALPHA_CONTEXT = {
        'name': 'ES_NewStrategy_DSP_LPBP_Combination_With_ManualHedge', # Global alpha name, which be used for load/save from DB
        'context': { # Strategy specific settings
            # These settings only applycable for alphas derived from StrategyAlpha strategy 
            # StrategyAlpha - is a classic EXO/SmartEXO based alpha
            'index_name': 'US.ES_ContFutEOD',      # Name of EXO index to trade         
            'costs_per_option': 3.0,
            'costs_per_contract': 3.0,
        },
        'wfo_params': {
            'window_type': 'rolling',  # Rolling window for IIS values: rolling or expanding
            'period': 'M',  # Period of rolling window 'M' - monthly or 'W' - weekly
            'oos_periods': 2,  # Number of months is OOS period
            'iis_periods': 20,
            # Number of months in IIS rolling window (only applicable for 'window_type' == 'rolling')
        },
        'wfo_optimizer_class': OptimizerGenetic, 
        'wfo_optimizer_class_kwargs': {
            'nbest_count': 3,
            'nbest_fitness_method': 'max',
            'population_size': 4, 
            'number_generations': 30, 
             'rand_seed': 1, # Uncomment this parameter to make genetic results repeatable
        },
        'wfo_opt_params': [
                ('Direction', [1]),
                ('LP order',[0,1,2,3,4,5,6,7,8,9,10,11,12,13]),
                ('LP freq',[0.7]), # 0 > f < 1
                ('BP order',[0,1,2,3,4,5,6,7,8,9]),
                ('BP start freq', [0.2986, 0.296, 0.297, 0.2944, 0.2995]),
                ('BP stop freq',   [0.01]),
                ('BP multiplier',[-10,-9,-8,-7,-6,-5,-4, -3,-2,-1,0,1,2,3,4,5,6,7,8,9,10] ),
                ('Rule index',   [3] ),                        
            ],
        'wfo_members_count': 1,
        'wfo_costs_per_contract': 0.0,
        'wfo_scoring_type': 'netprofit'
    }

# Run the alpha

In [ ]:
# DataManager is a core class of the framework
dm = DataManager(start_date=datetime.datetime(2016, 4, 1))
#dm = DataManager()

# Init alpha class and run
alpha = Strategy_DSP_LPBP_ManualPositionHedge(dm, **ALPHA_CONTEXT)    

In [ ]:
alpha.run()

# Equity

In [ ]:
alpha.stats['series']['equity'].plot()

In [ ]:
#alpha.save()

# Position holdings

#### Position hedge proof: alpha uses ES furues as primary position and options (from index position) as hedge

In [ ]:
for asset, pos_rec in alpha.position.get_net_position(pd.Timestamp('2017-05-02 12:45:00-07:00')).items():
    print(f'{asset} qty: {pos_rec[2]}')